# Retrieval Augmented Generation (RAG) with Langchain
*With IBM Granite Models*

### This notebook has moved.

Please find the notebook in the new `granite-snacks` repository: [RAG-with-Langchain](https://github.com/ibm-granite-community/granite-snack-cookbook/blob/main/recipes/RAG/RAG_with_Langchain.ipynb)